# Asistente Turístico de Tenerife (RAG + diálogo + `get_weather`)

Proyecto del módulo LLM (Máster IA, Cloud & DevOps).  
Objetivo: prototipo reproducible que responda preguntas sobre una guía de Tenerife usando **RAG**, mantenga **diálogo multiturturno** y realice una llamada externa simulada **`get_weather(fecha)`** con manejo de errores y log.


## Alcance
- Fuente: guía turística en PDF (Tenerife).
- RAG: chunking + embeddings + vector store (FAISS).
- LLM comercial (OpenAI).
- Diálogo con memoria de conversación.
- Function call: `get_weather(fecha: AAAA-MM-DD)` (simulada) + logging.

## Decisiones prácticas
- Extraeré texto **directamente del PDF**.
- Parámetros simples: `chunk_size=500`, `overlap=100`.
- Evaluación ligera: ejemplos de consultas, fuentes recuperadas y casos límite.


## Entorno y dependencias

Usaré un entorno de Python con las dependencias fijadas en `requirements.txt` para asegurar reproducibilidad.

- Instalación (desde el kernel seleccionado): `pip install -r requirements.txt`
- Variables privadas: crear `.env` con `OPENAI_API_KEY=...` (no se sube a Git).
- Paquetes clave: `langchain`, `langchain-openai`, `faiss-cpu`, `chromadb`, `pydantic`, `tiktoken`, `python-dotenv`, `pypdf`, `ipykernel`.

> Objetivo: tener el entorno preparado para extraer el PDF, crear *embeddings* y ejecutar el RAG.


In [12]:
%pip install -q -r requirements.txt

I0000 00:00:1757146462.759963   17920 fork_posix.cc:71] Other threads are currently calling into gRPC, skipping fork() handlers


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.19.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.3, but you have protobuf 6.32.0 which is incompatible.

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: /home/victordani/Escritorio/Pontia/7. ML-DL/pontia_modulo_ml_alumnos/env-pontia-ml/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Verificación de entorno y API
Cargo variables desde `.env` y hago dos *smoke tests*:
1) Llamada mínima al LLM.
2) Cálculo de *embeddings*.


In [7]:
from dotenv import load_dotenv; load_dotenv()
import os; assert os.getenv("GOOGLE_API_KEY"), "Falta GOOGLE_API_KEY en .env"

from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
# LLM
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0)
print("LLM:", llm.invoke("Responde literalmente: OK").content)  # esperado: OK
# Embeddings
emb = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")
print("Emb dim:", len(emb.embed_query("playa en Tenerife")))


LLM: OK
Emb dim: 768


## Lectura de la guía (PDF → texto)

Objetivo: cargar el PDF de Tenerife y obtener su texto en la variable `texto` para usarlo en el RAG.

- Ruta esperada: `data/TENERIFE.pdf`
- Librería: `pypdf` (ya instalada vía `requirements.txt`)
- Salida esperada: cuenta de caracteres y un preview del contenido


In [9]:
from pypdf import PdfReader
import os

pdf_path = "data/TENERIFE.pdf"
assert os.path.exists(pdf_path), "Copia TENERIFE.pdf dentro de la carpeta data/"

texto = "\n\n".join((p.extract_text() or "").strip() for p in PdfReader(pdf_path).pages)
print("Caracteres:", len(texto))
print(texto[:600], "…")


Caracteres: 16142
TENERIFE – LUGARES DE INTERÉS 
SITIOS QUE VER 
 
ZONA NORTE 
 
• Santa Cruz de Tenerife: 
Santa Cruz de Tenerife es la capital de la isla. Quizás la ruta a seguir si vais a Santa 
Cruz sería: 
- Aparcar en el aparcamiento del Parque Marítimo (ubicación). 
- Caminar por la Avenida Marítima hasta Plaza de España (ubicación). 
- Por el camino de la Avenida Marítima, ver el auditorio de Tenerife (ubicación). 
- Una vez llegados a Plaza España, callejear un poco (subir la Calle Castillo 
dirección Plaza Weyler - ubicación –; ir hacia el Parque García Sanabria - 
ubicación -; y bajar de nuevo hacia  …


## Chunking (trocear el texto)
Objetivo: dividir el texto de la guía en trozos manejables (≈500 caracteres con solapamiento) para poder buscar por significado.


In [10]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
chunks = splitter.split_text(texto)

print("Nº de chunks:", len(chunks))
print("Ejemplo del primer chunk:\n", chunks[0][:300], "…")

Nº de chunks: 49
Ejemplo del primer chunk:
 TENERIFE – LUGARES DE INTERÉS 
SITIOS QUE VER 
 
ZONA NORTE 
 
• Santa Cruz de Tenerife: 
Santa Cruz de Tenerife es la capital de la isla. Quizás la ruta a seguir si vais a Santa 
Cruz sería: 
- Aparcar en el aparcamiento del Parque Marítimo (ubicación). 
- Caminar por la Avenida Marítima hasta Plaz …


## Embeddings + vector store (FAISS)
Objetivo: convertir cada chunk en un vector semántico y guardarlos en un índice FAISS para búsqueda rápida.


In [13]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import FAISS

emb = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")
vs = FAISS.from_texts(chunks, emb)

# Comprobación rápida
probe = emb.embed_query("playa en Tenerife")
print("Vector store listo · nº chunks:", len(chunks), "· dim embedding:", len(probe))


Vector store listo · nº chunks: 49 · dim embedding: 768


## Búsqueda de prueba
Objetivo: comprobar que el índice devuelve trozos coherentes para una pregunta.


In [14]:
q = "¿Qué lugares imprescindibles visitar en Tenerife?"
docs = vs.similarity_search(q, k=3)

for i, d in enumerate(docs, 1):
    preview = d.page_content[:250].replace("\n", " ")
    print(f"[{i}] {preview}...\n")


[1] o Auditorio de Tenerife [vídeo - ubicación]      o Plaza de España [vídeo - ubicación]...

[2] También se encuentra al lado de esta playa el Papagayo Beach Club, el cual  ha sido elegido varios años como mejor beach club de España y al que tenéis  que ir sí o sí si queréis salir de fiesta por Tenerife [vídeo]. Podéis consultar las  fiestas del...

[3] Si queréis subir hasta el pico del Teide, podéis hacerlo desde aquí haciendo uso de  los teleféricos del Teide. Y, si queréis más info sobre el Teide, podéis ir al Centro de  Visitantes de El Portillo que es gratis.  Si os apetece, podéis subir de no...



## Metadatos para citar fuentes
Objetivo: reconstruir el índice FAISS guardando `chunk_id` para poder mostrar las fuentes recuperadas.


In [15]:
from langchain_community.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings

emb = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")
metas = [{"chunk_id": i} for i in range(len(chunks))]
vs = FAISS.from_texts(chunks, emb, metadatas=metas)

print("FAISS reconstruido con metadatos · chunks:", len(chunks))


FAISS reconstruido con metadatos · chunks: 49


## Chat conversacional (RAG)
Objetivo: mantener historial y responder usando los 3 trozos más relevantes del índice FAISS.

In [20]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import ConversationalRetrievalChain

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0)
retriever = vs.as_retriever(search_kwargs={"k": 3})

qa = ConversationalRetrievalChain.from_llm(
    llm, retriever=retriever, return_source_documents=True
)

chat_history = []

def chat(q):
    out = qa.invoke({"question": q, "chat_history": chat_history})
    chat_history.append((q, out["answer"]))
    print(out["answer"], "\n")

    for i, d in enumerate(out["source_documents"], 1):
        cid = d.metadata.get("chunk_id", "?")
        preview = d.page_content[:140].replace("\n"," ")
        print(f"[Fuente {i} · chunk {cid}] {preview}...")

# prueba
chat("Voy 2 días a Tenerife. ¿Qué no me puedo perder?")


Basándome en el texto proporcionado, para una visita de dos días a Tenerife no te puedes perder:

* **Auditorio de Tenerife:**  (Se incluye un vídeo y ubicación en el texto original)
* **Plaza de España:** (Se incluye un vídeo y ubicación en el texto original)
* **Papagayo Beach Club:**  (Se recomienda ir si quieres salir de fiesta, se incluye un vídeo y se recomienda reservar entrada con antelación).
* **Restaurante Fujiyama:** (Sushi en Santa Cruz, se incluye la ubicación).


El texto menciona otros lugares, pero indica que son muchos y que es recomendable organizar bien los días para aprovechar al máximo la visita. 

[Fuente 1 · chunk 2] o Auditorio de Tenerife [vídeo - ubicación]      o Plaza de España [vídeo - ubicación]...
[Fuente 2 · chunk 33] También se encuentra al lado de esta playa el Papagayo Beach Club, el cual  ha sido elegido varios años como mejor beach club de España y al...
[Fuente 3 · chunk 48] Recomendable reservar.  ▪ Restaurante Fujiyama (sushi en la zona de Santa

## Function call: `get_weather(fecha)`
Objetivo: definir una función **simulada** con Pydantic que valide `fecha` (AAAA-MM-DD), devuelva un pronóstico simple y **registre** cada intento en `logs.txt`.


In [22]:
from pydantic import BaseModel, ValidationError, field_validator
from datetime import datetime
import random, json

class GetWeatherArgs(BaseModel):
    fecha: str

    @field_validator("fecha")
    @classmethod
    def valida_fecha(cls, v: str):
        try:
            datetime.strptime(v, "%Y-%m-%d")
        except ValueError:
            raise ValueError("Formato inválido. Usa AAAA-MM-DD.")
        return v

def get_weather(fecha: str) -> dict:
    """Función simulada: genera un pronóstico reproducible y hace logging."""
    ts = datetime.now().isoformat(timespec="seconds")
    try:
        args = GetWeatherArgs(fecha=fecha)
        d = datetime.strptime(args.fecha, "%Y-%m-%d")
        random.seed(int(d.strftime("%Y%m%d")))  # reproducible por fecha
        estado = random.choice(["soleado", "parcialmente nublado", "nublado",
                                "lluvia ligera", "calima", "viento fuerte"])
        tmax = random.randint(22, 31)
        tmin = tmax - random.randint(3, 7)
        result = {"ok": True, "fecha": args.fecha, "estado": estado,
                  "tmin": tmin, "tmax": tmax}
        msg = f"[{ts}] get_weather ok fecha={args.fecha} -> {estado} {tmin}-{tmax}ºC\n"
    except ValidationError as e:
        result = {"ok": False, "error": e.errors()[0]["msg"]}
        msg = f"[{ts}] get_weather ERROR fecha={fecha} -> {result['error']}\n"
    finally:
        with open("logs.txt", "a", encoding="utf-8") as f:
            f.write(msg)
    return result

# Pruebas rápidas
print(get_weather("2025-09-10"))
print(get_weather("2025-99-99"))  # debe dar error de validación


{'ok': True, 'fecha': '2025-09-10', 'estado': 'nublado', 'tmin': 20, 'tmax': 26}
{'ok': False, 'error': 'Value error, Formato inválido. Usa AAAA-MM-DD.'}


## Integración de `get_weather` en el chat
Si la pregunta contiene palabras de “tiempo/clima” y una fecha `AAAA-MM-DD`, llamo a `get_weather`. Si falta fecha, aviso.

In [23]:
import re

def chat(q):
    ql = q.lower()
    # 1) Detección simple de intención "tiempo"
    if any(k in ql for k in ["tiempo", "clima", "weather", "pronóstico"]):
        m = re.search(r"\b\d{4}-\d{2}-\d{2}\b", q)
        if m:
            w = get_weather(m.group(0))
            if w["ok"]:
                ans = f"Pronóstico para {w['fecha']}: {w['estado']} ({w['tmin']}–{w['tmax']}ºC)"
            else:
                ans = f"Error en `get_weather`: {w['error']}"
            chat_history.append((q, ans))
            print(ans, "\n")
            return
        else:
            msg = "Para consultar el tiempo, indica una fecha en formato AAAA-MM-DD."
            chat_history.append((q, msg))
            print(msg, "\n")
            return

    # 2) RAG normal
    out = qa.invoke({"question": q, "chat_history": chat_history})
    chat_history.append((q, out["answer"]))
    print(out["answer"], "\n")
    for i, d in enumerate(out["source_documents"], 1):
        cid = d.metadata.get("chunk_id", "?")
        preview = d.page_content[:140].replace("\n"," ")
        print(f"[Fuente {i} · chunk {cid}] {preview}...")

# Pruebas mínimas
chat("¿Qué tiempo hará el 2025-09-10 en Tenerife?")
chat("¿Qué tiempo hará el 2025-99-99?")
chat("Dame 3 planes imprescindibles en Tenerife")


Pronóstico para 2025-09-10: nublado (20–26ºC) 

Error en `get_weather`: Value error, Formato inválido. Usa AAAA-MM-DD. 

Basándome en el texto proporcionado, tres planes imprescindibles en Tenerife serían:

1. Visitar el Teide:  Se menciona la posibilidad de subir al pico del Teide en teleférico y visitar el Centro de Visitantes de El Portillo.  Se destaca también la posibilidad de observar el cielo estrellado desde allí.

2. Explorar Costa Adeje: Se describe como la zona turística por excelencia de Tenerife.

3. Visitar Puerto de la Cruz: Se presenta como la zona turística del Norte de Tenerife,  sugiriendo un paseo desde el muelle hasta la Playa Martiánez. 

[Fuente 1 · chunk 2] o Auditorio de Tenerife [vídeo - ubicación]      o Plaza de España [vídeo - ubicación]...
[Fuente 2 · chunk 29] Si queréis subir hasta el pico del Teide, podéis hacerlo desde aquí haciendo uso de  los teleféricos del Teide. Y, si queréis más info sobre...
[Fuente 3 · chunk 15] tendréis poca playa (especialmen

## Validación de `get_weather` (3 invocaciones correctas + 1 con error)


In [24]:
# Tres fechas válidas
chat("¿Qué tiempo hará el 2025-09-10 en Tenerife?")
chat("¿Qué tiempo hará el 2025-12-25 en Tenerife?")
chat("¿Qué tiempo hará el 2026-01-05 en Tenerife?")

# Un caso con error (fecha inválida)
chat("¿Qué tiempo hará el 2025-99-99 en Tenerife?")


Pronóstico para 2025-09-10: nublado (20–26ºC) 

Pronóstico para 2025-12-25: soleado (15–22ºC) 

Pronóstico para 2026-01-05: viento fuerte (24–28ºC) 

Error en `get_weather`: Value error, Formato inválido. Usa AAAA-MM-DD. 



## Persistencia del índice (FAISS)
Objetivo: guardar el índice en disco y comprobar que se recarga correctamente.


In [26]:
# Guarda el índice en una carpeta local
vs.save_local("vs_tenerife")
print("Índice guardado en ./vs_tenerife")


Índice guardado en ./vs_tenerife


In [27]:
from langchain_community.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings

emb = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")
vs2 = FAISS.load_local("vs_tenerife", emb, allow_dangerous_deserialization=True)

# Prueba rápida
docs = vs2.similarity_search("mejores playas de Tenerife", k=2)
print("Recarga OK · resultados:", len(docs))
print(docs[0].page_content[:160], "…")

# Usaremos vs2 en adelante
vs = vs2


Recarga OK · resultados: 2
También se encuentra al lado de esta playa el Papagayo Beach Club, el cual 
ha sido elegido varios años como mejor beach club de España y al que tenéis 
que ir  …
